In [1]:
%pip install -r ./requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pyspark.sql import SparkSession
from io import StringIO
import boto3
import paramiko

In [3]:
def create_spark_session():
        """
        Create and return a Spark session configured for AWS S3 access.
        Returns:
            SparkSession: Configured Spark session object.
        """
        aws_profile_name = "default"

        spark = SparkSession.builder \
                            .appName("SplitColumnExample") \
                            .config(
                                "spark.jars.packages",
                                "org.apache.hadoop:hadoop-aws:3.3.4,"
                                "com.amazonaws:aws-java-sdk-bundle:1.12.262"
                            ) \
                            .config(
                                "spark.hadoop.fs.s3a.aws.credentials.provider",
                                "com.amazonaws.auth.profile.ProfileCredentialsProvider"
                            ) \
                            .config("spark.hadoop.fs.s3a.profile", aws_profile_name) \
                            .getOrCreate()
        
        return spark

In [4]:
def get_s3_data():
    """
    Function to list and print S3 object keys in a specified bucket and prefix.
    Returns:
        str: S3 path of the last object found in the specified bucket and prefix.
    """

    s3 = boto3.client('s3')
    bucket_name = "data-challenge-loadsmart-raw"
    bucket_prefix = "data/"

    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=bucket_prefix)

    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                print(obj['Key'])
                data_path = f"s3a://{bucket_name}/{obj['Key']}"

    return data_path

In [5]:
spark = create_spark_session()
data_path = get_s3_data()

:: loading settings :: url = jar:file:/home/leonardooliveira/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/leonardooliveira/.ivy2/cache
The jars for the packages stored in: /home/leonardooliveira/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4fc8f61c-8458-4308-9666-60663c8ced67;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 321ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number|

data/2025_data_challenge_loadsmart.csv


In [6]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv(data_path)
df.show()

25/12/01 23:34:19 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/12/01 23:34:27 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
25/12/01 23:34:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+--------------------+----------------+----------------+----------------+----------------+----------------+----------+------------+--------+-------+--------------+--------------+----------------+-----------+------------------------+---------------+------------+-------------------------+---------------------------+-----------------------+-----------------------+-------------------------+-------------------------+-------------------------+-----------------------+----------------+---------------+------------------------+-------------------------+------------------+
|loadsmart_id|                lane|      quote_date|       book_date|     source_date|     pickup_date|   delivery_date|book_price|source_price|     pnl|mileage|equipment_type|carrier_rating|sourcing_channel|vip_carrier|carrier_dropped_us_count|   carrier_name|shipper_name|carrier_on_time_to_pickup|carrier_on_time_to_delivery|carrier_on_time_overall|pickup_appointment_time|delivery_appointment_time|has_mobile_app_t

25/12/01 23:34:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: loadsmart_id, lane, quote_date, book_date, source_date, pickup_date, delivery_date, book_price, source_price, pnl, mileage, equipment_type, carrier_rating, sourcing_channel, vip_carrier, carrier_dropped_us_count, carrier_name, shipper_name, carrier_on_time_to_pickup, carrier_on_time_to_delivery, carrier_on_time_overall, pickup_appointment_time, delivery_appointment_time, has_mobile_app_tracking, has_mobile_app_tracking, has_macropoint_tracking, has_edi_tracking, contracted_load, load_booked_autonomously, load_sourced_autonomously, load_was_cancelled
 Schema: loadsmart_id, lane, quote_date, book_date, source_date, pickup_date, delivery_date, book_price, source_price, pnl, mileage, equipment_type, carrier_rating, sourcing_channel, vip_carrier, carrier_dropped_us_count, carrier_name, shipper_name, carrier_on_time_to_pickup, carrier_on_time_to_delivery, carrier_on_time_overall, pickup_appointment_t

In [7]:
df.count()

5361

In [28]:
def convert_dataframe_to_csv_and_send_via_sftp(df):
    """
    Convert a Spark DataFrame to CSV format and send it via SFTP to a specified server
    Args:
        df (DataFrame): Spark DataFrame to be converted and sent.
    """
    
    private_key = paramiko.RSAKey.from_private_key_file("/home/leonardooliveira/.ssh/id_rsa")

    ssh_client = paramiko.SSHClient()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh_client.connect(hostname="localhost", port=2222, pkey=private_key)

    sftp_client = ssh_client.open_sftp()

    csv_data = StringIO()
    df.toPandas().to_csv(csv_data, index=False)
    csv_data.seek(0)

    with sftp_client.open('./sftp_root/data_challenge.csv', 'w') as remote_file:
        remote_file.write(csv_data.getvalue())

    sftp_client.close()
    ssh_client.close()

In [30]:
convert_dataframe_to_csv_and_send_via_sftp(df)

25/12/02 00:52:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: loadsmart_id, lane, quote_date, book_date, source_date, pickup_date, delivery_date, book_price, source_price, pnl, mileage, equipment_type, carrier_rating, sourcing_channel, vip_carrier, carrier_dropped_us_count, carrier_name, shipper_name, carrier_on_time_to_pickup, carrier_on_time_to_delivery, carrier_on_time_overall, pickup_appointment_time, delivery_appointment_time, has_mobile_app_tracking, has_mobile_app_tracking, has_macropoint_tracking, has_edi_tracking, contracted_load, load_booked_autonomously, load_sourced_autonomously, load_was_cancelled
 Schema: loadsmart_id, lane, quote_date, book_date, source_date, pickup_date, delivery_date, book_price, source_price, pnl, mileage, equipment_type, carrier_rating, sourcing_channel, vip_carrier, carrier_dropped_us_count, carrier_name, shipper_name, carrier_on_time_to_pickup, carrier_on_time_to_delivery, carrier_on_time_overall, pickup_appointment_t